### How to speeeeeeeed up your Python scientific code?

* Think about vectorization in numpy
* Think about vectorization harder
* Really, dude, create a vectorized version

### Example 1

A sum of matrix elements

In [1]:
import numpy as np


SIZE = 10**3
x = np.random.rand(SIZE, SIZE)

%timeit x.sum()

550 µs ± 45 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [2]:
def my_sum(x):
    sum_x = 0
    for i in range(len(x)):
        for j in range(len(x[i,:])):
            sum_x += x[i, j]
    return sum_x

%timeit my_sum(x)

249 ms ± 8.52 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Example 2

Pairwise distances between rows in two matrices

In [3]:
def pairwise_dists(x, y):
    """  
    Computing pairwise distances

    Parameters
    ----------
    x : numpy.ndarray, shape=(M, D)
    y : numpy.ndarray, shape=(N, D)

    Returns
    -------
    numpy.ndarray, shape=(M, N)
     The Euclidean distance between each pair of
     rows between `x` and `y`.
    """
    raise NotImplementedError()

### Your ideas??

In [4]:
def looped_pairwise_dists(x, y):
    """  
    Computing pairwise distances using for-loops

    Parameters
    ----------
    x : numpy.ndarray, shape=(M, D)
    y : numpy.ndarray, shape=(N, D)

    Returns
    -------
    numpy.ndarray, shape=(M, N)
     The Euclidean distance between each pair of
     rows between `x` and `y`.
    """
    # `dists[i, j]` will store the Euclidean
    # distance between  `x[i]` and `y[j]`
    dists = np.empty((x.shape[0], y.shape[0]))

    for i, row_x in enumerate(x):     # loops over rows of `x`
        for j, row_y in enumerate(y): # loops over rows of `y`
            # Subtract corresponding entries of the rows,
            # squares each difference, and then sums them. This
            # exactly matches our equation for Euclidean
            # distance (we will do the square root later)
            dists[i, j] = np.sum((row_x - row_y)**2)

    # we still need to take the square root of
    # each of our numbers
    return np.sqrt(dists)

### Numpy broadcasting!

Rules of Broadcasting:

* the aligned dimensions have the same size
* one of the dimensions has a size of 1

The two arrays are broadcast-compatible if either of these conditions are satisfied for each pair of aligned dimensions.

### A very quick reminders:

     array-1:         8
     array-2: 5 x 2 x 8
     result-shape: 5 x 2 x 8
     --------

     array-1:     5 x 2
     array-2: 5 x 4 x 2
     result-shape: INCOMPATIBLE
     --------

     array-1: 5 x 1 x 3 x 2
     array-2:     9 x 1 x 2
     result-shape: 5 x 9 x 3 x 2
     --------

     array-1: 1 x 3 x 2
     array-2:     8 x 2
     result-shape: INCOMPATIBLE
     --------

     array-1: 2 x 1
     array-2:     1
     result-shape: 2 x 1
     --------

### So what about pairwise distance??

In [5]:
def numpy_pairwise_dists(x, y):
    """  
    Computing pairwise distances using for-loops

    Parameters
    ----------
    x : numpy.ndarray, shape=(M, D)
    y : numpy.ndarray, shape=(N, D)

    Returns
    -------
    numpy.ndarray, shape=(M, N)
     The Euclidean distance between each pair of
     rows between `x` and `y`.
    """
    # x[:, np.newaxis] is (M, 1, D)
    # y[np.newaxis] is (1, N, D)
    # x[:, np.newaxis] - y[np.newaxis] is (M, N, D)
    # sum over the last axis
    return np.sqrt(np.sum((x[:, np.newaxis] - y[np.newaxis])**2, axis=2))

In [6]:
M = 100
N = 500
D = 64

x = np.random.rand(M, D).astype(np.float32)
y = np.random.rand(N, D).astype(np.float32)

print('looped version')
%timeit looped_pairwise_dists(x, y)

print('numpy version')
%timeit numpy_pairwise_dists(x, y)

looped version
300 ms ± 13.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
numpy version
10.2 ms ± 623 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Example 3

When are you one hundred percent sure to fail with vectorization?

* **Recursive functions with unknown number of iterations!**

### What to do if you are fail with vectorization?

There are three ways

* make a parallel computation (multithreading, multiprocessing, joblib)
* optimize your Python byte code (Numba, Py-Py)
* write your custom C/C++ lib and use it directly in your Python (Cython, Boost)
    

![](./figures/numba_love.png)

## Numba

* an open source JIT compiler that translates a subset of Python and NumPy code into fast machine code.
* could be install from conda, pip or source

## Numba limitations

* Numba compiles individual functions. Not whole programs like e.g. PyPy
    
    
* Numba supports a subset of Python. Some dict/list/set support, but not mixed types for keys or values


* Numba supports a subset of Numpy

![](./figures/numba_works.png)

### Example 2

How to compute pairwise distance between vectors in two matrices

In [7]:
import numba
from numba import jit
from numba import float32


@jit(signature_or_function=float32[:,:](float32[:,:], float32[:,:]), nopython=True)
def jit_pairwise_dists_looped(x, y):
    """ 
    Computing pairwise distances using for-loops

    Parameters
    ----------
    x : numpy.ndarray, shape=(M, D)
    y : numpy.ndarray, shape=(N, D)

    Returns
    -------
    numpy.ndarray, shape=(M, N)
     The Euclidean distance between each pair of
     rows between `x` and `y`.
    """
    dists = np.zeros((x.shape[0], y.shape[0]), dtype=np.float32)

    for i in range(len(x)):
        for j in range(len(y)):
            dists[i, j] = np.sum((x[i,:] - y[j,:])**2)

    return np.sqrt(dists)

In [8]:
@numba.jit
def f(n):
    return n * ['bla']

f(3)

['bla', 'bla', 'bla']

In [9]:
@numba.jit(nopython=True)
def f(n):
    return n * ['bla']

f(3)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mInvalid use of * with parameters (int64, list(const('bla')))
Known signatures:
 * (int64, int64) -> int64
 * (int64, uint64) -> int64
 * (uint64, int64) -> int64
 * (uint64, uint64) -> uint64
 * (float32, float32) -> float32
 * (float64, float64) -> float64
 * (complex64, complex64) -> complex64
 * (complex128, complex128) -> complex128
 * parameterized[0m
[0m[1m[1] During: typing of intrinsic-call at <ipython-input-9-58cb10259b7f> (3)[0m
[1m
File "<ipython-input-9-58cb10259b7f>", line 3:[0m
[1mdef f(n):
[1m    return n * ['bla']
[0m    [1m^[0m[0m

This is not usually a problem with Numba itself but instead often caused by
the use of unsupported features or an issue in resolving types.

To see Python/NumPy features supported by the latest release of Numba visit:
http://numba.pydata.org/numba-doc/dev/reference/pysupported.html
and
http://numba.pydata.org/numba-doc/dev/reference/numpysupported.htm

In [10]:
M = 100
N = 500
D = 64

x = np.random.rand(M, D).astype(np.float32)
y = np.random.rand(N, D).astype(np.float32)

print('looped version')
%timeit looped_pairwise_dists(x, y)

print('numpy version')
%timeit numpy_pairwise_dists(x, y)

print('numba version')
%timeit numpy_pairwise_dists(x, y)

looped version
299 ms ± 30.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
numpy version
10.2 ms ± 889 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
numba version
10.4 ms ± 1.6 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Ideal solution is scipy.spatial.distance.cdist!

Let's look how close we are

In [11]:
from scipy.spatial.distance import cdist

print("scipy implementation")
%timeit cdist(x, y)

scipy implementation
2.77 ms ± 249 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
x = np.random.rand(1_000_000)

@numba.jit
def f(x):
    return np.cos(x)**2 + np.sin(x)**2

f(x)
print('sequential version')
%timeit f(x)

@numba.jit(parallel=True)
def f(x):
    return np.cos(x)**2 + np.sin(x)**2

f(x)
print('parallel version')
%timeit f(x)

sequential version
16.2 ms ± 1.33 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
parallel version
8.24 ms ± 332 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Cython

* a superset of the Python
* designed to give C-like performance
* code in Cython must be compiled
* some parts of Scipy, pandas and scikit-learn are written in Cython

*The fundamental nature of Cython can be summed up as follows: Cython is Python with C data types.*

In [14]:
%load_ext cython

In [15]:
%%cython
import numpy as np
cimport numpy as np

cpdef np.ndarray[float, ndim=2] cython_pairwise_dists(
    np.ndarray[float, ndim=2] x,
    np.ndarray[float, ndim=2] y
):
    
    cdef np.ndarray dists = np.zeros((x.shape[0], y.shape[0]))
    cdef int i, j = 0
    
    for i in range(len(x[0])):  
        for j in range(len(y[0])): 
            dists[i, j] = np.sum((x[i,:] - y[j,:])**2)
            
    return np.sqrt(dists)

In [16]:
M = 100
N = 500
D = 64

x = np.random.rand(M, D).astype(np.float32)
y = np.random.rand(N, D).astype(np.float32)

%timeit cython_pairwise_dists(x, y)

27.4 ms ± 4.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://i.stack.imgur.com/cl4pb.png")